Import all required libraries

In [11]:
from die import Die
from grid import grid
from numpy import array, ndarray
from collections import defaultdict

Define the backtracking function

In [12]:
def backtrack(row: int,
              col: int,
              die: Die,
              score: int,
              turn: int,
              matrix: array,
              visited_cells: defaultdict[tuple[int, int], set[str]]) -> bool:
    """
    Returns true if a satisfiable path has been found.
    """
    # Remember the value of this cell
    c: int = matrix[row][col]

    # Remember whether the top-face was set during this function call
    top_face_number_set_here: bool = False

    if turn > 0:
        if not die.top_face_has_number():
            # Since c = s + t x n, n exists iff t divides c - s
            if (c - score) % turn == 0:
                die.set_top_face_number((c - score) // turn)
                top_face_number_set_here = True
            else:
                # No integer will work as the die-face
                return False

    # Given the number on the top-face, does the equation still hold?
    if c != score + turn * die.get_top_face_number():
        # We only enter this conditional if the die-face number was not set
        # in this function call, because if it was, n would have been chosen
        # by construction to satisfy the equation.
        #
        # Thus, we don't delete the die-face number here, because this face might be
        # valid, just that the right path does not include this cell.
        return False

    # Check if you've already visited this cell with the same orientation
    orientation: str = die.serialize()

    if orientation in visited_cells[(row, col)]:
        return False  # You've made no progress
        
    # Check: Are we on the last square?
    if row == 0 and col == len(grid[0]) - 1:
        return True
        
    visited_cells[(row, col)].add(orientation)
    
    # Check: Does rolling up work?
    die.tip_up()
    if row > 0 and backtrack(row - 1, col, die, c, turn + 1, matrix, visited_cells):
        matrix[row - 1][col] = 0
        return True
    die.tip_down()
    
    # Check: Does rolling down work?
    die.tip_down()
    if row + 1 < len(matrix) and backtrack(row + 1, col, die, c, turn + 1, matrix, visited_cells):
        matrix[row + 1][col] = 0
        return True
    die.tip_up()

    # Check: Does rolling left work?
    die.tip_left()
    if col > 0 and backtrack(row, col - 1, die, c, turn + 1, matrix, visited_cells):
        matrix[row][col - 1] = 0
        return True
    die.tip_right()

    # Check: Does rolling right work?
    die.tip_right()
    if col + 1 < len(matrix[0]) and backtrack(row, col + 1, die, c, turn + 1, matrix, visited_cells):
        matrix[row][col + 1] = 0
        return True
    die.tip_left()

    # Everything failed. If you had set a die-face here, reset it.
    if top_face_number_set_here:
        die.set_top_face_number(0)

    # Deregister yourself from visited_cells
    visited_cells[(row, col)].remove(orientation)
    return False


Print the matrix before starting for comparison's sake

In [13]:
print(grid)

[[ 57  33 132 268 492 732]
 [ 81 123 240 443 353 508]
 [186  42 195 704 452 228]
 [ -7   2 357 452 317 395]
 [  5  23  -4 592 445 620]
 [  0  77  32 403 337 452]]


Run the backtracker

In [14]:
visited_cells: dict[tuple[int, int], set[str]] = defaultdict(set)
print("Success" if backtrack(row=len(grid) - 1, col=0, die=Die(), score=0, turn=0, matrix=grid, visited_cells=visited_cells) else "Failure")
print(len(visited_cells))

Failure
25


Set the cells to 0 if the die has visited it before

In [15]:
print(grid)

[[ 57  33 132 268 492 732]
 [ 81 123 240 443 353 508]
 [186  42 195 704 452 228]
 [ -7   2 357 452 317 395]
 [  5  23  -4 592 445 620]
 [  0  77  32 403 337 452]]
